In [160]:
import PyPDF2 as p2
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import pandas as pd
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pickle
from bs4 import BeautifulSoup

import json
import numpy as np
import re

import fitz
import math

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [161]:
df_outlines = pd.read_csv("F:\Environmental Baseline Data\Version 4 - Final\Indices\Index 2 - PDFs for Major Projects with ESAs.csv")
 
len(df_outlines)
df_outlines[20:30].head(5)

#define the paths of the PDF scrapped 
"F:/Environmental Baseline Data/Version 4 - Final/PDF/" + df_outlines.iloc[3]['DataID_pdf'] 
path = "F:/Environmental Baseline Data/Version 4 - Final/PDF/"
#'F:/Environmental Baseline Data/Version 3/Data/PDF/'
#pdfs = os.listdir(path)

1729

,Unnamed: 0,file_name,application_name,esa_folder_link,DocumentID,DataID,esa_download_link,DataID_pdf,Section,Application filing date,...,Company Name,Consultant Name,Status in GIS,RTS English Title = Project name,Regulatory Instruments Issued,Pipeline Named in GIS,Application title short,Link to Application,Link to Decision,Location
20,20,C03258-3 EIS Volume 3- Biophysical Baseline - ...,2004-10-07 - Application for the Construction ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A7A3J7,3891472,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3891472.pdf,52,2004-10-07,...,Imperial Oil Resources Ventures Limited (Macke...,Imperial Oil,NaN,NaN,NaN,NaN,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/6...,NaN
21,21,C03258-4 EIS Volume 3- Biophysical Baseline - ...,2004-10-07 - Application for the Construction ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A7A3J8,3891895,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3891895.pdf,52,2004-10-07,...,Imperial Oil Resources Ventures Limited (Macke...,Imperial Oil,NaN,NaN,NaN,NaN,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/6...,NaN
22,22,C03258-5 EIS Volume 3- Biophysical Baseline - ...,2004-10-07 - Application for the Construction ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A7A3J9,3891896,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3891896.pdf,52,2004-10-07,...,Imperial Oil Resources Ventures Limited (Macke...,Imperial Oil,NaN,NaN,NaN,NaN,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/6...,NaN
23,23,C03258-6 EIS Volume 3- Biophysical Baseline - ...,2004-10-07 - Application for the Construction ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A7A3K0,3892113,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3892113.pdf,52,2004-10-07,...,Imperial Oil Resources Ventures Limited (Macke...,Imperial Oil,NaN,NaN,NaN,NaN,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/6...,NaN
24,24,C03258-7 EIS Volume 3- Biophysical Baseline - ...,2004-10-07 - Application for the Construction ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A7A3K1,3891801,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3891801.pdf,52,2004-10-07,...,Imperial Oil Resources Ventures Limited (Macke...,Imperial Oil,NaN,NaN,NaN,NaN,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/6...,NaN


'F:/Environmental Baseline Data/Version 4 - Final/PDF/269018.pdf'

# Extracting The Features

In [162]:
def count_word_in_str(long_str, key):
    count = 0
    for word in long_str.split():
        if key in word:
            count = count + 1
    return(count)
count_word_in_str("vibudh rocks dh dh ddh","dh")

4

In [163]:
def count_words_in_str(long_str, keys):
    count = 0
    for word in long_str.split():
        for key in keys:
            if key in word:
                count = count + 1
    return(count)
count_words_in_str("vibudh rocks dh dh ddh",["dh","vi"])

5

In [164]:
from string import digits

s = 'abc123def456ghi789ersuit834678 dhfuaiwhbui34tr234 zero0'
remove_digits = str.maketrans('', '', digits)
res = s.translate(remove_digits)
res

'abcdefghiersuit dhfuaiwhbuitr zero'

In [165]:
def area_of_imgblocks(imgblocks):
    sum_areas = 0
    for imgblock in imgblocks:
        block_area = 0.5*imgblock['width']* imgblock['height']
        sum_areas = sum_areas + block_area
    return(sum_areas)

In [166]:
def area_of_imgblocks(imgblocks):
    sum_areas = 0
    for imgblock in imgblocks:
        d = math.sqrt(math.pow((imgblock['bbox'][0]-imgblock['bbox'][2]),2) + math.pow((imgblock['bbox'][1]-imgblock['bbox'][3]),2))
        area = 0.5*d*d
        sum_areas = sum_areas + area
    return(sum_areas)



In [167]:
def extract_features(dataIDs):
    words_in_page = []
    
    scale = []
    km_kilometers = []
    m = []
    metres = []
    scale_grp = []
    
    legend = []
    
    figure = []
    mapp = []
    alignment_sheet = []
    sheet = []
    figure_grp = []
    
    north = []
    n = []
    north_grp = []
    
    dataID_pageNo = []
    
    i = 0
    count = 0
    
    No_of_images = []
    Area_of_images = []
    cnt = 0
    for dataID in dataIDs:
        pdf_path = 'F:/Environmental Baseline Data/Version 4 - Final/PDF/' + str(dataID) +'.pdf'

        doc = fitz.open(pdf_path)
        img_ = dict()
        
    
        for page in doc:  # iterate through the pages
            cnt = cnt + 1
            d = page.getText("dict")
            blocks = d["blocks"]
            imgblocks = [b for b in blocks if b["type"] == 1]
            No_of_images.append(len(imgblocks))
            Area_of_images.append(area_of_imgblocks(imgblocks))
        
        
        
        
        
        
        i = i+1
        pages_file = []
        pk_path =  'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_' + str(dataID) + '.pkl'
        print("File Starting: {}. PDF {} out of {}".format(dataID, i, len(dataIDs)))
        with open(pk_path, 'rb') as f:
            data = pickle.load(f)
            soup = BeautifulSoup(data['content'], 'lxml')
            pages = soup.find_all('div', attrs={'class': 'page'})
            for b, p in enumerate(pages):
                
                
                p = str(p).replace('<p>', '').replace('</p>','').replace(".",'').replace(",",'').replace('"','').lower()
                p = p.translate(remove_digits)
                words_lst = p.split()
                word_count = 0
                big_words = ""
                words = ""
                for word in words_lst:
                    words  = words + " " + word
                    if len(word) > 3:
                        word_count = word_count + 1
                        big_words = big_words + " " + word
                        
                words_in_page.append(word_count)
                
                
                sc_grp = 0 
                if "scale" in big_words:
                    scale.append(count_word_in_str(big_words, "scale"))
                    sc_grp = 1
                else:
                    scale.append(0)
                    
                if ("kilometre" in big_words or "kilometer" in big_words or "km " in p):
                    km_kilometers.append(count_words_in_str(p, ["kilometre", "kilometer", "km "]))
                    sc_grp = 1
                else:
                    km_kilometers.append(0)
                    
                if("m " in p):
                    m.append(count_word_in_str(p, "m "))
                else:
                    m.append(0)
                    
                if("metre" in big_words or "meter" in big_words):
                    metres.append(count_words_in_str(big_words, ["meter","metre"]))
                    sc_grp = 1
                else:
                    metres.append(0)
                    
                if sc_grp > 0:
                    scale_grp.append(1)
                else:
                    scale_grp.append(0)
                    
                
                
                if "legend" in big_words:
                    legend.append(count_word_in_str(big_words,"legend"))
                else:
                    legend.append(0)
                
                    
                    
                fig_grp = 0
                if "figure" in big_words:
                    figure.append(count_word_in_str(big_words,"figure"))
                    fig_grp = 1
                else:
                    figure.append(0)
                    
                if "map " in p:
                    mapp.append(count_word_in_str(p,"map "))
                    fig_grp = 1
                else:
                    mapp.append(0)
                    
                    
                if "alignment sheet" in big_words:
                    alignment_sheet.append(1)
                    fig_grp = 1
                else:
                    alignment_sheet.append(0)
                    
                if "sheet" in big_words:
                    sheet.append(count_word_in_str(big_words,"sheet"))
                    fig_grp = 1
                else:
                    sheet.append(0)
                
                if fig_grp > 0:
                    figure_grp.append(1)
                else:
                    figure_grp.append(0)
                
                
                    
                   
                if "north" in big_words:
                    north.append(count_word_in_str(big_words, "north"))
                    no_grp = 1
                else:
                    north.append(0)
                    
                if "n" in p:
                    n.append(count_word_in_str(p, " n "))
                    no_grp = 1
                else:
                    n.append(0)
                
                
                             
                dataID_pageNo.append(str(dataID) + "_" + str(b+1))
                             
    #print("Total Number of pages processed: {}".format(count))   
    return scale, km_kilometers, m, metres, scale_grp, legend, figure, mapp, alignment_sheet, sheet, figure_grp, north, n, words_in_page, dataID_pageNo, No_of_images, Area_of_images

In [168]:
"F:\Environmental Baseline Data\Version 4 - Final\PDF\895015.pdf"

'F:\\Environmental Baseline Data\\Version 4 - Final\\PDF\\895015.pdf'

# Prepare Training Data 

In [169]:
DataIDHand = [895015, 2392922, 2445549, 2967854, 2968069, 268712, 
              486221, 500633, 555093, 631608, 684494, 2758927,
              3891802,
             2813701]
Pages = [range(1,11), [1], range(1,4), [3,4],range(1,13), [3,4,5,8,9,10,14,15,24,25,26], 
         range(1,5), [5,9], [6,9,33,34], [6,11,38], [12,13,14], [9], 
         [33, 34, 35, 89, 90, 91, 92, 93, 100, 146, 147, 148, 149, 153, 154, 159, 160, 161, 162, 165, 166, 169, 170, 173, 174, 178, 181, 182, 184, 185, 188, 189], 
        [40, 92, 95, 143, 170, 180, 216, 217, 218, 219]]

#fetching featuresfor the pages of the PDF Files
scale, km_kilometers, m, metres, scale_grp, legend, figure, mapp, alignment_sheet, sheet, figure_grp, north, n, words_in_page, dataID_pageNo, No_of_images, Area_of_images = extract_features(DataIDHand) 

Y_ali = []
for i in range(len(DataIDHand)):
    for j in Pages[i]:
        s = str(DataIDHand[i]) + "_" +str(j)
        Y_ali.append(s)
        #print(s)

File Starting: 895015. PDF 1 out of 14
File Starting: 2392922. PDF 2 out of 14
File Starting: 2445549. PDF 3 out of 14
File Starting: 2967854. PDF 4 out of 14
File Starting: 2968069. PDF 5 out of 14
File Starting: 268712. PDF 6 out of 14
File Starting: 486221. PDF 7 out of 14
File Starting: 500633. PDF 8 out of 14
File Starting: 555093. PDF 9 out of 14
File Starting: 631608. PDF 10 out of 14
File Starting: 684494. PDF 11 out of 14


mupdf: invalid page object


File Starting: 2758927. PDF 12 out of 14
File Starting: 3891802. PDF 13 out of 14
File Starting: 2813701. PDF 14 out of 14


In [170]:
def get_Y_values(dataIDs):
    Y_class = []
    for dataID in dataIDs:
        
        pk_path =  'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_' + str(dataID) + '.pkl'
        with open(pk_path, 'rb') as f:
            data = pickle.load(f)
            soup = BeautifulSoup(data['content'], 'lxml')
            pages = soup.find_all('div', attrs={'class': 'page'})
            for b, p in enumerate(pages):
                s = str(dataID) + "_" +str(b+1)
                if s in Y_ali:
                    Y_class.append(1)
                    #print(s)
                else:
                    Y_class.append(0)
                    
    return(Y_class)
                            
Y_class = get_Y_values(DataIDHand)

In [171]:
len(Y_class)
len(dataID_pageNo)
len(No_of_images)
len(Area_of_images)
len(Y_ali)

897

897

897

897

98

# Preparing Data for Trainihng 

In [172]:
Xdf_dataset = pd.DataFrame({'scale' : scale, 
                           'km_kilometers' : km_kilometers, 
                           'm' : m, 
                           'metres' : metres, 
                           'scale_grp' : scale_grp, 
                           'legend' : legend, 
                           'figure' : figure, 
                           'mapp' : mapp, 
                           'alignment_sheet' : alignment_sheet, 
                           'sheet' : sheet, 
                           'figure_grp' : figure_grp, 
                           'north' : north, 
                           'n' : n, 
                           'words_in_page' : words_in_page,
                            'No_of_images' : No_of_images, 
                            'Area_of_images' : Area_of_images,
                           'dataID_pageNo' : dataID_pageNo,
                           #'Y_class' : Y_class
                           })
Xdf_dataset.head()


,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images,dataID_pageNo
0,1,0,0,0,1,1,0,0,1,4,1,0,0,272,5,1.931526e+06,895015_1
1,1,0,0,0,1,1,0,0,1,4,1,0,0,284,5,1.931526e+06,895015_2
2,1,0,0,1,1,1,0,0,1,4,1,0,0,263,5,1.931526e+06,895015_3
3,1,0,0,0,1,1,0,0,1,4,1,0,0,244,5,1.931526e+06,895015_4
4,1,0,0,0,1,1,0,0,1,4,1,0,0,237,5,1.931526e+06,895015_5


In [173]:
XYdf_dataset = pd.DataFrame({'scale' : scale, 
                           'km_kilometers' : km_kilometers, 
                           'm' : m, 
                           'metres' : metres, 
                           'scale_grp' : scale_grp, 
                           'legend' : legend, 
                           'figure' : figure, 
                           'mapp' : mapp, 
                           'alignment_sheet' : alignment_sheet, 
                           'sheet' : sheet, 
                           'figure_grp' : figure_grp, 
                           'north' : north, 
                           'n' : n, 
                             'No_of_images' : No_of_images, 
                            'Area_of_images' : Area_of_images,
                           'dataID_pageNo' : dataID_pageNo,
                           'Y_class' : Y_class
                           })


XYdf_dataset.to_csv("F:/Environmental Baseline Data/Version 4 - Final/Support files/Random Forest/AM2 - Features_of_PDF_Pages.csv")

In [174]:
Ydf_dataset = pd.DataFrame({'Y_class' : Y_class})
Ydf_dataset.head()

,Y_class
0,1
1,1
2,1
3,1
4,1


In [175]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xdf_dataset,
                                                    Ydf_dataset,
                                                    test_size = 0.2,
                                                    random_state = 8)
X_train.shape
X_test.shape
y_train.shape
y_test.shape

(717, 17)

(180, 17)

(717, 1)

(180, 1)

In [176]:
#Feature Scaling
"""
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) """

'\nfrom sklearn.preprocessing import StandardScaler\n\nsc = StandardScaler()\nX_train = sc.fit_transform(X_train)\nX_test = sc.transform(X_test) '

In [177]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=20, random_state=0)
#n_estimators - Number of trees in the forest 
#random_state - Controls both the randomness of the bootstrapping of the samples used when building 
              # trees (if bootstrap=True) and the sampling of the features to consider when looking 
              # for the best split at each node (if max_features < n_features)
# Ref -> https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html


regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
#y_pred.shape
#y_pred
y_predict = []
for y in y_pred:
    if y > 0.5:
        y_predict.append(1)
    else:
        y_predict.append(0)
#y_predict


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=20,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [178]:
f_importance = regressor.feature_importances_
feature = []

for col in Xdf_dataset:
    feature.append(col)
    
df_f_importance = pd.DataFrame({'Feature_Name' :  feature, 
                                'Importance':  f_importance})
df_f_importance

,Feature_Name,Importance
0,scale,0.012180
1,km_kilometers,0.048504
2,m,0.000000
3,metres,0.008270
4,scale_grp,0.000000
5,legend,0.000000
6,figure,0.005852
7,mapp,0.000000
8,alignment_sheet,0.000000
9,sheet,0.000000


In [179]:
# https://stackabuse.com/random-forest-algorithm-with-python-and-scikit-learn/

In [180]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict))
print(accuracy_score(y_test, y_predict))

[[160   1]
 [  2  17]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       161
           1       0.94      0.89      0.92        19

    accuracy                           0.98       180
   macro avg       0.97      0.94      0.95       180
weighted avg       0.98      0.98      0.98       180

0.9833333333333333


In [181]:
XYRdf_dataset = X_test
XYRdf_dataset['Y_test'] = y_test
XYRdf_dataset['y_score'] = y_pred
XYRdf_dataset['y_predict'] = y_predict
#XYRdf_dataset['dataID_pageNo'] = dataID_pageNo
len(XYRdf_dataset)
XYRdf_dataset.head()


XYRdf_dataset.to_csv("F:/Environmental Baseline Data/Version 4 - Final/Support files/Random Forest/AM2 - Results_Training_Set.csv")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

180

,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images,dataID_pageNo,Y_test,y_score,y_predict
841,0,0,0,0,0,0,0,0,0,0,1,15,0,94,2,9683.890181,2813701_164,0,0.0,0
728,0,0,0,5,1,0,1,0,0,0,1,1,0,281,2,9683.903780,2813701_51,0,0.0,0
643,0,0,0,0,0,0,1,0,0,0,1,0,0,22,1,307168.527520,3891802_170,1,1.0,1
366,0,0,0,0,0,0,0,0,0,0,0,0,0,124,0,0.000000,2758927_25,0,0.0,0
311,0,0,0,0,0,0,0,0,0,0,0,1,0,444,0,0.000000,684494_79,0,0.0,0


# Testing with new files

In [182]:
DataIDHand = [549192, 
              894925,
              2392985,
              2967770,
              2546079,
              3581159, 
             269018,
             2541470,
              2968352
             ]

#DataIDHand = [2813701]
scale, km_kilometers, m, metres, scale_grp, legend, figure, mapp, alignment_sheet, sheet, figure_grp, north, n, words_in_page, dataID_pageNo, No_of_images, Area_of_images = extract_features(DataIDHand) 

File Starting: 549192. PDF 1 out of 9
File Starting: 894925. PDF 2 out of 9
File Starting: 2392985. PDF 3 out of 9
File Starting: 2967770. PDF 4 out of 9
File Starting: 2546079. PDF 5 out of 9
File Starting: 3581159. PDF 6 out of 9
File Starting: 269018. PDF 7 out of 9
File Starting: 2541470. PDF 8 out of 9
File Starting: 2968352. PDF 9 out of 9


In [183]:
Xdf_dataset = pd.DataFrame({'scale' : scale, 
                           'km_kilometers' : km_kilometers, 
                           'm' : m, 
                           'metres' : metres, 
                           'scale_grp' : scale_grp, 
                           'legend' : legend, 
                           'figure' : figure, 
                           'mapp' : mapp, 
                           'alignment_sheet' : alignment_sheet, 
                           'sheet' : sheet, 
                           'figure_grp' : figure_grp, 
                           'north' : north, 
                           'n' : n, 
                           'words_in_page' : words_in_page,
                            'No_of_images' : No_of_images, 
                            'Area_of_images' : Area_of_images,
                           'dataID_pageNo' : dataID_pageNo,
                           #'Y_class' : Y_class
                           })
Xdf_dataset.head()

#df_dataset.to_csv("F:/Environmental Baseline Data/Version 4 - Final/Support files/Features_of_PDF_Pages.csv")

,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images,dataID_pageNo
0,1,0,0,1,1,0,0,0,0,0,0,0,0,268,1,1282.183284,549192_1
1,1,0,0,1,1,0,0,0,0,0,0,1,0,330,1,1282.183988,549192_2
2,0,0,0,0,0,0,0,0,0,0,0,0,0,221,1,1282.183284,549192_3
3,0,0,0,0,0,0,0,0,0,0,0,0,0,219,1,1282.183988,549192_4
4,0,0,0,0,0,0,0,0,0,0,0,0,0,224,1,1282.183284,549192_5


In [184]:
Ydf_dataset = regressor.predict(Xdf_dataset)
#y_pred.shape
#y_pred
Ydf_dataset_predict = []
for y in Ydf_dataset:
    if y > 0.5:
        Ydf_dataset_predict.append(1)
    else:
        Ydf_dataset_predict.append(0)
#y_predict

In [185]:
XYRTdf_dataset = Xdf_dataset
#XYRdf_dataset['Y_test'] = y_test
XYRTdf_dataset['y_score'] = Ydf_dataset
XYRTdf_dataset['y_predict'] = Ydf_dataset_predict
#XYRTdf_dataset['dataID_pageNo'] = dataID_pageNo
#len(XYRdf_dataset)
#XYRdf_dataset.head()


XYRTdf_dataset.to_csv("F:/Environmental Baseline Data/Version 4 - Final/Support files/Random Forest/AM2 - Results_Test_Set.csv")